In [2]:
% matplotlib inline
import pylab as plt
import numpy as np
import networkx as nx
import pandas as pd
import operator
import os
import re
from glob import glob
import os
os.chdir('/home/jagust/dino/surf/')
import util as util
import modularity as md
from scipy import stats
import operator

def sort_by_val(d):
    return sorted(d.items(), key = operator.itemgetter(1))

def find_next_node(r, g):
    """ Find the next node to be removed in the simulation
    
    Parameters
    ----------
        r : set of removed nodes
        
    Returns
    -------
        node : next node to be removed
    """
    nodes = g.nodes()
    distance = {}
    for node in nodes:
        if node in r:
            continue
        distance[node] = 0.0
        for removed in r:
            distance[node] += nx.shortest_path_length(g, node, removed, 'weight') 
        sorted_distances = sort_by_val(distance)
    return sorted_distances[0][0]

def run_simulation_dampening(fmat, pnet, seed, stop):
    ''' Run lesion simulation
    Parameters
    ----------
    fnet : functional network
    pnet : PiB correlation network
    seed : beginning of simulated lesioning
    stop : terminate simulation after removing stop number of nodes
    
    Returns
    -------
    mods : tracks modularity of functional network as nodes are removed
    effs : tracks efficiencies of functional network as nodes are removed
    
    Notes:
        -Partition of functional network is found using Newman Spectral
        -Efficiency is defined as the inverse of the average shortest path length
            of the functional network
    '''
    # Create Graph
    fnet, c = util.threshold_adjacency_matrix(fmat, cost=0.20)
    
    # Find partition
    ind = md.newman_partition(fnet, max_div=np.inf).index
    
    # Initial global metrics
    mods, effs = [], []
    part = md.GraphPartition(fnet, index=ind)
    mods.append(part.modularity())
    effs.append(find_eff(fnet))
    
    
    #removed = {seed}
    
    while len(removed) < stop:
        rem = find_next_node(removed, pnet)
        removed.add(rem)
        fnet.remove_edges_from(fnet.edges(rem))
        
        mods.append(md.GraphPartition(fnet, index=ind).modularity())
        effs.append(find_eff(fnet))
    
    return mods, effs

In [16]:
ll = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]
arr = np.array(ll)

In [17]:
arr[0] = arr[0] * 0.25

In [18]:
arr

array([[ 0.25,  0.5 ,  0.75],
       [ 4.  ,  5.  ,  6.  ],
       [ 7.  ,  8.  ,  9.  ]])